<a href="https://colab.research.google.com/github/yukinaga/object_detection/blob/main/section_3/03_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習
RetinaNetで、物体の領域を出力する`regression_head`も訓練対象に加えてみましょう。  
モデルを構築するコードに、追記を行なってください。

## 各設定

In [ ]:
import torch
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import draw_bounding_boxes

import numpy as np
import matplotlib.pyplot as plt
import math

# インデックスを物体名に変換
index2name = [
    "person",
    "bird",
    "cat",
    "cow",
    "dog",
    "horse",
    "sheep",
    "aeroplane",
    "bicycle",
    "boat",
    "bus",
    "car",
    "motorbike",
    "train",
    "bottle",
    "chair",
    "diningtable",
    "pottedplant",
    "sofa",
    "tvmonitor",
]
print(index2name)

# 物体名をインデックスに変換
name2index = {}
for i in range(len(index2name)):
    name2index[index2name[i]] = i
print(name2index)

## ターゲットを整える関数

In [ ]:
def arrange_target(target):
    objects = target["annotation"]["object"]
    box_dics = [obj["bndbox"] for obj in objects]
    box_keys = ["xmin", "ymin", "xmax", "ymax"]

    # バウンディングボックス
    boxes = []
    for box_dic in box_dics:
        box = [int(box_dic[key]) for key in box_keys]
        boxes.append(box)
    boxes = torch.tensor(boxes)

    # 物体名
    labels = [name2index[obj["name"]] for obj in objects]  # 物体名はインデックスに変換
    labels = torch.tensor(labels)

    dic = {"boxes":boxes, "labels":labels}
    return dic

## データセットの読み込み

In [ ]:
dataset_train=torchvision.datasets.VOCDetection(root="./VOCDetection/2012",
                                                year="2012",image_set="train",
                                                download=True,
                                                transform=transforms.ToTensor(),
                                                target_transform=transforms.Lambda(arrange_target)
                                                )

dataset_test=torchvision.datasets.VOCDetection(root="./VOCDetection/2012",
                                                year="2012",image_set="val",
                                                download=True,
                                                transform=transforms.ToTensor(),
                                                target_transform=transforms.Lambda(arrange_target)
                                                )

## DataLoaderの設定

In [ ]:
data_loader_train =  DataLoader(dataset_train, batch_size=1, shuffle=True)
data_loader_test =  DataLoader(dataset_test, batch_size=1, shuffle=True)

## ターゲットの表示

In [ ]:
def show_boxes(image, boxes, names):
    drawn_boxes = draw_bounding_boxes(image, boxes, labels=names)

    plt.figure(figsize = (16,16))
    plt.imshow(np.transpose(drawn_boxes, (1, 2, 0)))  # チャンネルを一番後ろに
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # ラベルとメモリを非表示に
    plt.show()

dataiter = iter(data_loader_train)  # イテレータ
image, target = next(dataiter)  # バッチを取り出す
print(target)

image = image[0]
image = (image*255).to(torch.uint8)  # draw_bounding_boxes関数の入力は0-255

boxes = target["boxes"][0]

labels = target["labels"][0]
names = [index2name[label.item()] for label in labels]

show_boxes(image, boxes, names)

# モデルの構築
以下のセルのコードに追記を行い、物体領域の座標を出力する`regression_head`のパラメータも訓練可能にしましょう。  
PyTorchの公式ドキュメントに記載されている、RetinaNetのコードを参考にしましょう。  
https://pytorch.org/vision/stable/_modules/torchvision/models/detection/retinanet.html#retinanet_resnet50_fpn

In [ ]:
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)

num_classes=len(index2name)+1  # 分類数: 背景も含めて分類するため1を加える
num_anchors = model.head.classification_head.num_anchors  # アンカーの数

# 分類数を設定
model.head.classification_head.num_classes = num_classes

# 分類結果を出力する層の入れ替え
cls_logits = torch.nn.Conv2d(256, num_anchors*num_classes, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)  # RetinaNetClassificationHeadクラスより
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))  # RetinaNetClassificationHeadクラスより
model.head.classification_head.cls_logits = cls_logits  # 層の入れ替え

# 全てのパラメータを更新不可に
for p in model.parameters():
    p.requires_grad = False

# classification_headのパラメータを更新可能に
for p in model.head.classification_head.parameters():
    p.requires_grad = True

# regression_headのパラメータを更新可能に
# ------- 以下にコードを書く -------


# ------- ここまで -------

model.cuda()  # GPU対応

## 訓練

In [ ]:
# 最適化アルゴリズム
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9)

model.train()  # 訓練モード
epochs = 3
for epoch in range(epochs):
    for i, (image, target) in enumerate(data_loader_train):
        image = image.cuda()  # GPU対応

        boxes = target["boxes"][0].cuda()
        labels = target["labels"][0].cuda()
        target = [{"boxes":boxes, "labels":labels}]  # ターゲットは辞書を要素に持つリスト

        loss_dic = model(image, target)
        loss = sum(loss for loss in loss_dic.values())  # 誤差の合計を計算

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i%100 == 0:  # 100回ごとに経過を表示
            print("epoch:", epoch,  "iteration:", i,  "loss:", loss.item()) 

## 訓練したモデルの使用

In [ ]:
dataiter = iter(data_loader_test)  # イテレータ
image, target = next(dataiter)  # バッチを取り出す

image = image.cuda()  # GPU対応

model.eval()
predictions = model(image)
print(predictions)

image = (image[0]*255).to(torch.uint8).cpu() # draw_bounding_boxes関数の入力は0-255
boxes = predictions[0]["boxes"].cpu()
labels = predictions[0]["labels"].cpu().detach().numpy()
labels = np.where(labels>=len(index2name), 0, labels)  # ラベルが範囲外の場合は0に
names = [index2name[label.item()] for label in labels]

print(names)
show_boxes(image, boxes, names)

## スコアによる選別

In [ ]:
boxes = []
names = []
for i, box in enumerate(predictions[0]["boxes"]):
    score = predictions[0]["scores"][i].cpu().detach().numpy()
    if score > 0.5:  # スコアが0.5より大きいものを抜き出す
        boxes.append(box.cpu().tolist())
        label = predictions[0]["labels"][i].item()
        if label >= len(index2name):  # ラベルが範囲外の場合は0に
            label = 0
        name = index2name[label]
        names.append(name)
boxes = torch.tensor(boxes)

show_boxes(image, boxes, names)

# 解答例
以下は、どうしても手がかりがないときのみ参考にしましょう。

In [ ]:
model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)

num_classes=len(index2name)+1  # 分類数: 背景も含めて分類するため1を加える
num_anchors = model.head.classification_head.num_anchors  # アンカーの数

# 分類数を設定
model.head.classification_head.num_classes = num_classes

# 分類結果を出力する層の入れ替え
cls_logits = torch.nn.Conv2d(256, num_anchors*num_classes, kernel_size=3, stride=1, padding=1)
torch.nn.init.normal_(cls_logits.weight, std=0.01)  # RetinaNetClassificationHeadクラスより
torch.nn.init.constant_(cls_logits.bias, -math.log((1 - 0.01) / 0.01))  # RetinaNetClassificationHeadクラスより
model.head.classification_head.cls_logits = cls_logits  # 層の入れ替え

# 全てのパラメータを更新不可に
for p in model.parameters():
    p.requires_grad = False

# classification_headのパラメータを更新可能に
for p in model.head.classification_head.parameters():
    p.requires_grad = True

# regression_headのパラメータを更新可能に
# ------- 以下にコードを書く -------
for p in model.head.regression_head.parameters():
    p.requires_grad = True
# ------- ここまで -------

model.cuda()  # GPU対応